# SQL II: Grouping and Joining

_September 21, 2020_

Agenda:
- Practice with Grouping in SQLite
- Practice with different types of Joins in SQLite

<img src = "https://media.giphy.com/media/3oKIPnAiaMCws8nOsE/giphy.gif">

In [1]:
import sqlite3
import pandas as pd
import seaborn as sns
import numpy as np

## Part I. Grouping
Just like pandas, Sql support different types of grouping statements for performing aggregate functions and allow us to calculate various statistics for data.

Syntax:
`SELECT (*) FROM tables GROUP BY column`

In [2]:
# instantiate some data
auto = pd.read_csv('auto-mpg.csv')
tips = sns.load_dataset('tips')
small_quiz = pd.DataFrame({"students":["Raizel","Mitch","Eric"],
                          "projects":[1,2,1],
                          "grades":np.random.randint(80,100,3)})
small_grades = pd.DataFrame({"students":["Mike W","Mitch","Mike R"],
                            "quiz_score":np.random.randint(0,10,3)})

In [3]:
conn = sqlite3.connect('aggregate.db')
c = conn.cursor()

In [4]:
# insert these datasets into our db
auto.to_sql('auto', con=conn)
tips.to_sql('tips', con=conn)
small_grades.to_sql('small_grade', con=conn)
small_quiz.to_sql('small_quiz',con=conn)

In [25]:
# for the tips df/table, get the total number of man and woman
c.execute("""SELECT COUNT(sex) FROM tips GROUP BY sex;""").fetchall()
# pd_query_1 = tips.sex.value_counts()
# pd_query_1

[(87,), (157,)]

In [26]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


Syntax: `SELECT ___ FROM ___ GROUP BY ___` 

In [ ]:
# in sql 
query = None
c.execute(query).fetchall()

In [29]:
# using pandas, select the average amount tipped for time
tips.groupby(['time'])['tip'].mean()

time
Lunch     2.728088
Dinner    3.102670
Name: tip, dtype: float64

In [32]:
# use sql to recreate this query
query = """SELECT time, AVG(tip) FROM tips GROUP BY time;"""
c.execute(query).fetchall()

[('Dinner', 3.102670454545454), ('Lunch', 2.7280882352941176)]

We can also use `GROUP BY` with certain conditions, in conjunction with filtering and ordering.

You can use :

`SELECT ___ FROM ___ WHERE ___ GROUP BY ___ `

`SELECT ___ FROM ___ GROUP BY ___ HAVING ____ `

In [40]:
# use pandas, get the average total bills for a party larger than 2 for every size of party
tips[tips['size'] > 2].groupby(['size']).total_bill.mean()

# using pandas

size
3    23.277632
4    28.613514
5    30.068000
6    34.830000
Name: total_bill, dtype: float64

In [41]:
# use sql to recreate this query
query = """SELECT size, AVG(total_bill) FROM tips WHERE size > 2 GROUP BY size;"""
c.execute(query).fetchall()

[(3, 23.27763157894737), (4, 28.61351351351351), (5, 30.068), (6, 34.83)]

In [46]:
# your turn - use pandas, get the maximum value of total bill for female non smoker
tips.groupby(['sex', 'smoker']).max()['total_bill']['Female']['No']
# using group by?

35.83

In [50]:
# how to get this without using group by?
tips.loc[(tips.sex == 'Female') & (tips.smoker == 'No')].max()['total_bill']

35.83

In [56]:
# use sql to recreate this query
query = """SELECT MAX(total_bill) FROM tips WHERE sex = 'Female' AND smoker = 'No';"""
c.execute(query).fetchall()

[(35.83,)]

In [53]:
# your turn - use pandas, get the average tip values for different size groups, and show top 2
# most amount of average tips
tips.groupby(['size'])['tip'].mean().sort_values(ascending = False).head(2)

size
6    5.225000
4    4.135405
Name: tip, dtype: float64

In [63]:
# use sql 
query = """SELECT size, AVG(tip) FROM tips GROUP BY size DESC;"""
c.execute(query).fetchall()

OperationalError: no such column: DESC

## Part II. Join

Based on this diagram, how would you explain the different types of joins?

<img src = 'sql-joins.png' width = 400>

In [64]:
print(small_grades)
print(small_quiz)

students  quiz_score
0   Mike W           1
1    Mitch           4
2   Mike R           1
  students  projects  grades
0   Raizel         1      80
1    Mitch         2      99
2     Eric         1      94


In [ ]:
# left join - join small quiz with small grades, on student name
small_grades.merge(small_quiz, left_on = 'students', right_on = 'students', how = "left")

# if i run this command, what would happen? how many observations/record?

In [70]:
# sql 
query = """SELECT * FROM small_grades JOIN small_quiz ON small_grades.students = small_quiz.students"""
c.execute(query).fetchall()

OperationalError: no such table: small_grades

In [ ]:
# inner join and get the student's grades
small_grades.merge(small_quiz, on = 'students')

In [73]:
query = """SELECT * FROM small_quiz LEFT JOIN small_grade ON small_grade.students = small_quiz.students"""
c.execute(query).fetchall()

[(0, 'Raizel', 1, 80, None, None, None),
 (1, 'Mitch', 2, 99, 1, 'Mitch', 4),
 (2, 'Eric', 1, 94, None, None, None)]

In [74]:
# your turn - use right join and get all entries
small_grades.merge(small_quiz, on= 'students', how= 'right')
# pandas

,students,quiz_score,projects,grades
0,Mitch,4.0,2,99
1,Raizel,NaN,1,80
2,Eric,NaN,1,94


In [77]:
# sql 
query1 = """SELECT * FROM small_grade LEFT JOIN small_quiz ON small_quiz.students = small_grade.students;"""
c.execute(query1).fetchall()

[(0, 'Mike W', 1, None, None, None, None),
 (1, 'Mitch', 4, 1, 'Mitch', 2, 99),
 (2, 'Mike R', 1, None, None, None, None)]

### Level up 
1. Subqueries (including anti-joins)
2. Get median (using OFFSET)

In [ ]:
# anti-join :get all the entries that are in the left table but not in the right table 
query = """SELECT left_table.id FROM `left table` WHERE left_table.id not in (SELECT id FROM right_table) """

In [ ]:
# get median - use offset 
query = """SELECT col FROM table ORDER BY col LIMIT 1 OFFSET (SELECT COUNT(*) FROM MyTable) / 2"""